# Обучение сиамской сети для различия двух объектов

In [1]:
import albumentations as A
import cv2
import matplotlib as plt
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch import Generator
from torch.utils.data import ConcatDataset, DataLoader, random_split

from src.config import MOT20_EXT_FIRST_AXIS_MEAN, MOT20_EXT_SECOND_AXIS_MEAN
from src.data import MOT20ExtDataset
from src.train import display_images, get_statistics
from src.transforms import get_norm_transform, get_resize_transform

/home/nick/reidentification/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nick/reidentification/.conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/nick/reidentification/.conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


## Загрузка данных

In [2]:
resize_transform = get_resize_transform((MOT20_EXT_FIRST_AXIS_MEAN, MOT20_EXT_SECOND_AXIS_MEAN)) 
norm_transform = get_norm_transform()
transform = A.Compose([resize_transform, norm_transform])

In [3]:
dataset01 = MOT20ExtDataset('data/MOT20_ext/train/MOT20-01/', transform=transform)
dataset02 = MOT20ExtDataset('data/MOT20_ext/train/MOT20-02/', transform=transform)
dataset03 = MOT20ExtDataset('data/MOT20_ext/train/MOT20-03/', transform=transform)
dataset05 = MOT20ExtDataset('data/MOT20_ext/train/MOT20-05/', transform=transform)

In [4]:
dataset = ConcatDataset([dataset01, dataset02, dataset03, dataset05])

In [5]:
len(dataset)

135721

### Создание даталоадеров

In [6]:
TEST_PROPROTION = 0.2
VAL_PROPORTION = 0.15
TRAIN_PROPORTION = 1 - TEST_PROPROTION - VAL_PROPORTION
sum([TEST_PROPROTION, VAL_PROPORTION, TRAIN_PROPORTION])

1.0

In [7]:
generator = torch.manual_seed(0)

In [8]:
train_set, val_set, test_set = random_split(
    dataset, [TRAIN_PROPORTION, VAL_PROPORTION, TEST_PROPROTION], generator=generator)

In [9]:
BATCH_SIZE = 64

In [10]:
train_loader = DataLoader(
    train_set,
    shuffle=True,
    batch_size=BATCH_SIZE,
    drop_last=True,
    generator=generator,
    
)

val_loader = DataLoader(
    val_set,
    shuffle=True,
    batch_size=BATCH_SIZE,
    drop_last=True,
    generator=generator
)

test_loader = DataLoader(
    test_set,
    shuffle=True,
    batch_size=BATCH_SIZE,
    drop_last=True,
    generator=generator
)

In [11]:
len(train_loader), len(val_loader), len(test_loader)

(1378, 318, 424)

### Расчет статистик

In [12]:
loader = DataLoader(
    dataset,
    shuffle=True,
    batch_size=BATCH_SIZE,
    drop_last=True,
    generator=generator,
)

In [13]:
# get_statistics(loader)

### Проверка отображения

## Обучение